In [1]:
from mp_api.client.mprester import MPRester

In [2]:
from pymatgen.core.periodic_table import Element

In [4]:
API_KEY = "fqPPo7Czb5mkbFh8mltlZd0I33csuKv0"
mpr = MPRester(API_KEY)

In [19]:
#下载所有含有Li，O的结构
docs=mpr.summary.search(elements=['Li', 'O'], all_fields=False, fields=['material_id', 'composition_reduced', 'formula_pretty', 'formula_anonymous',
                                                                        'symmetry', 'chemsys','density', 
                                                                        'structure', 'energy_above_hull', 'band_gap', 
                                                                        'efermi', 'is_gap_direct', 'theoretical'])

Retrieving SummaryDoc documents:   0%|          | 0/17066 [00:00<?, ?it/s]

In [20]:
ele_list = docs[0].chemsys.split('-')
# a = ["O", "C"]
def if_tm(element_list):
    # 返回体系中是否有过渡金属,输入应尬是一个列表，里面是各个元素的字符串
    for ele in element_list:
        if 'd' in Element(ele).electronic_structure:
            tm = 1
            break
        else:
            tm = 0
    return bool(tm)

def tm_number(composition):
    #返回结构中过渡金属的化学计量，输入应为上面爬的数据中的composition_reduced属性
    #前置库：pymatgen.periodic_table.Element
    tm = []
    ele_dict= composition.as_dict()
    ele = list(ele_dict.keys())
    for i in ele: 
        if 'd' in Element(i).electronic_structure:
            tm.append(i)
    stoihiometry_tm = 0
    for ele in tm:
        stoihiometry_tm += int(ele_dict[ele])
    return stoihiometry_tm



In [21]:
#选择具有过渡金属且锂原子更多的结构
rich_lithium_docs = []
for i in docs:
    ele_list = i.chemsys.split('-')
    stoichiometry_li = int(i.composition_reduced.as_dict()["Li"])
    stoichiometry_tm = tm_number(i.composition_reduced)
    if if_tm(ele_list)==True and stoichiometry_li > stoichiometry_tm:
        rich_lithium_docs.append(i)

In [22]:
rich_lithium_docs[0]

MPDataDoc<SummaryDoc>(
composition_reduced=Comp: Li7 Mn4 Co1 O12,
formula_pretty='Li7Mn4CoO12',
formula_anonymous='AB4C7D12',
chemsys='Co-Li-Mn-O',
density=3.936521933949453,
symmetry=SymmetryData(crystal_system=<CrystalSystem.tri: 'Triclinic'>, symbol='P1', number=1, point_group='1', symprec=0.1, version='1.16.2'),
material_id=MPID(mp-1175256),
structure=Structure Summary
Lattice
    abc : 2.974031 5.088312604306559 15.041613085759952
 angles : 96.22129275400442 93.63750082568448 103.5607925488666
 volume : 219.04110366464872
      A : 2.974031 0.0 0.0
      B : -1.193092 4.946459 0.0
      C : -0.954296 -1.906965 14.889692
    pbc : True True True
PeriodicSite: Li (0.2875, 2.7607, 1.4049) [0.3654, 0.5945, 0.0944]
PeriodicSite: Li (1.7665, 3.2069, 3.5605) [0.9678, 0.7405, 0.2391]
PeriodicSite: Li (0.5219, 3.7763, 6.2252) [0.6806, 0.9246, 0.4181]
PeriodicSite: Li (0.3257, -0.6844, 8.6553) [0.3304, 0.0857, 0.5813]
PeriodicSite: Li (-1.0081, -0.1930, 11.1704) [0.0021, 0.2502, 0.7502]
Per

In [23]:
# 把上面的富锂材料结构存为poscar形式
for struct in rich_lithium_docs:
    struct.structure.to(fmt='poscar', filename='rich_lithium_POSCAR/'+struct.formula_pretty+'.vasp')

In [25]:
# 把富锂材料结构存为cif形式
for struct in rich_lithium_docs:
    struct.structure.to('rich_lithium_cif/'+struct.formula_pretty+'.cif')

In [46]:
rich_lithium_docs[5].symmetry.symbol

'C2/c'

In [41]:
import pandas as pd
import numpy as np

In [58]:
data=[rich_lithium_docs[0].formula_pretty, rich_lithium_docs[0].material_id.string, rich_lithium_docs[0].symmetry.symbol, 
     rich_lithium_docs[0].density, rich_lithium_docs[0].energy_above_hull, rich_lithium_docs[0].band_gap, rich_lithium_docs[0].efermi,
     rich_lithium_docs[0].is_gap_direct, rich_lithium_docs[0].theoretical]
df.loc[len(df)]=data
df

,formula,mp-id,crystal system,density,energy above hull,band gap,efermi,is gap direct,theoretical
0,Li7Mn4CoO12,mp-1175256,P1,3.936522,0.059534,0.7477,2.613981,False,True
1,Li7Mn4CoO12,mp-1175256,P1,3.936522,0.059534,0.7477,2.613981,False,True


In [62]:
#把需要的数据存入到dataframe中
df = pd.DataFrame(columns=['formula', 'mp-id', 'crystal system', 'density', 'energy above hull', 'band gap', 'efermi', 'is gap direct', 'theoretical'])
for doc in rich_lithium_docs:
    data_item = [doc.formula_pretty, doc.material_id.string, doc.symmetry.symbol, 
     doc.density, doc.energy_above_hull, doc.band_gap, doc.efermi, doc.is_gap_direct, i.theoretical]
    df.loc[len(df)]=data_item

len(df)

5979

In [63]:
df[:20]

,formula,mp-id,crystal system,density,energy above hull,band gap,efermi,is gap direct,theoretical
0,Li7Mn4CoO12,mp-1175256,P1,3.936522,0.059534,0.7477,2.613981,False,True
1,Li7Mn5O12,mp-1175344,P-1,3.847434,0.048052,0.6739,2.664749,False,True
2,Li9Mn2Co5O16,mp-1176207,P1,4.125582,0.128365,0.0000,2.489637,False,True
3,Li2Si3NiO8,mp-863431,P2_1,2.804621,0.093111,3.4899,1.861893,False,True
4,Li2VSiO5,mp-18860,P4/nmm,3.063754,0.000000,2.3621,2.458561,False,True
5,Li3Bi(PO4)2,mp-760650,C2/c,4.216148,0.060688,3.1798,2.051327,False,True
6,Li9Mn2Co5O16,mp-1175757,C2,4.199492,0.071088,0.8130,2.416212,False,True
7,Li3P3(WO6)2,mp-777465,R-3,4.385320,0.036371,3.6739,2.035883,False,True
8,Li2CoP2O7,mp-697726,P-1,3.067371,0.034159,3.0108,1.688148,False,True
9,Li3MnSiCO7,mp-772668,P2_1/m,2.773203,0.041342,0.8431,1.712843,False,True


In [64]:
# 数据写入到lithium_rich_data.csv
df.to_csv('./lithium_rich_data.csv')

In [29]:
for i in rich_lithium_docs:
    
    
    
json_data = json.dumps(json_predata, ensure_ascii=False, indent=4)
with open("data.json", "w", encoding='utf-8') as file:
    file.write(json_data)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[29], line 3
      1 json_predata = []
      2 for i in rich_lithium_docs:
----> 3     dict1 = i.as_dict()
      4     json_predata.append(dict1)
      7 json_data = json.dumps(json_predata, ensure_ascii=False, indent=4)

File D:\language\anaconda\lib\site-packages\mp_api\client\core\client.py:797, in BaseRester._generate_returned_model.<locals>.new_getattr(self, attr)
    792     raise AttributeError(
    793         f"'{attr}' data is available but has not been requested in 'fields'."
    794         " A full list of unrequested fields can be found in `fields_not_requested`."
    795     )
    796 else:
--> 797     raise AttributeError(
    798         f"{self.__class__.__name__!r} object has no attribute {attr!r}"
    799     )

AttributeError: 'MPDataDoc' object has no attribute 'as_dict'
